![CMCC](http://cmcc.ufabc.edu.br/images/logo_site.jpg)
# **Spark + Python = PySpark**

#### Esse notebook introduz os conceitos b�sicos do Spark atrav�s de sua interface com a linguagem Python. Como aplica��o inicial faremos o cl�ssico examplo de contador de palavras . Com esse exemplo � poss�vel entender a l�gica de programa��o funcional para as diversas tarefas de explora��o de dados distribu�dos.

#### Para isso utilizaremos o livro texto [Trabalhos completos de William Shakespeare](http://www.gutenberg.org/ebooks/100) obtidos do  [Projeto Gutenberg](http://www.gutenberg.org/wiki/Main_Page).  Veremos que esse mesmo algoritmo pode ser empregado em textos de qualquer tamanho.

#### ** Esse notebook cont�m:  **
#### *Parte 1:* Criando uma base RDD e RDDs de tuplas
#### *Parte 2:* Manipulando RDDs de tuplas
#### *Parte 3:* Encontrando palavras �nicas e calculando m�dias
#### *Parte 4:* Aplicar contagem de palavras em um arquivo
#### *Parte 5:* Similaridade entre Objetos
#### Para os exerc�cios � aconselh�vel consultar a documenta��o da [API do PySpark](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD)

### ** Part 1: Criando e Manipulando RDDs **

In [1]:
from pyspark import SparkContext
sc =SparkContext()

#### ** (1a) Criando uma base RDD **
#### Podemos criar uma base RDD de diversos tipos e fonte do Python com o comando `sc.parallelize(fonte, particoes)`, sendo fonte uma vari�vel contendo os dados (ex.: uma lista) e particoes o n�mero de parti��es para trabalhar em paralelo.

In [2]:
ListaPalavras = ['gato', 'elefante', 'rato', 'rato', 'gato']
palavrasRDD = sc.parallelize(ListaPalavras, 4)
print type(palavrasRDD)

<class 'pyspark.rdd.RDD'>


In [3]:
# EXERCICIO
def Plural(palavra):
    """Adds an 's' to `palavra`.

    Args:
        palavra (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
    novaPalavra = palavra + 's'
    return novaPalavra

print Plural('gato')

gatos


In [4]:
help(Plural)

Help on function Plural in module __main__:

Plural(palavra)
    Adds an 's' to `palavra`.
    
    Args:
        palavra (str): A string.
    
    Returns:
        str: A string with 's' added to it.



In [5]:
assert Plural('rato')=='ratos', 'resultado incorreto!'
print 'OK'

OK


#### ** (1c) Aplicando a fun��o ao RDD **
#### Transforme cada palavra do nosso RDD em plural usando [map()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.map) 

#### Em seguida, utilizaremos o comando  [collect()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.collect) que retorna a RDD como uma lista do Python.

In [6]:
# EXERCICIO --Só consegui fazer com lambda
pluralRDD = palavrasRDD.map(lambda x: (x + 's'))
print pluralRDD.collect()

['gatos', 'elefantes', 'ratos', 'ratos', 'gatos']


In [7]:
assert pluralRDD.collect()==['gatos','elefantes','ratos','ratos','gatos'], 'valores incorretos!'
print 'OK'

OK


#### ** (1d) Utilizando uma fun��o `lambda` **
#### Repita a cria��o de um RDD de plurais, por�m utilizando uma fun��o lambda.

#### ** Nota: ** utilize o comando `collect()` apenas quando tiver certeza de que a lista caber� na mem�ria. Para gravar os resultados de volta em arquivo texto ou base de dados utilizaremos outro comando.

In [8]:
# EXERCICIO
pluralLambdaRDD = palavrasRDD.map(lambda x: (x + 's'))
print pluralLambdaRDD.collect()

['gatos', 'elefantes', 'ratos', 'ratos', 'gatos']


In [9]:
assert pluralLambdaRDD.collect()==['gatos','elefantes','ratos','ratos','gatos'], 'valores incorretos!'
print 'OK'

OK


#### ** (1e) Tamanho de cada palavra **
#### Agora use `map()` e uma fun��o `lambda` para retornar o n�mero de caracteres em cada palavra. Utilize `collect()` para armazenar o resultado em forma de listas na vari�vel destino.

In [10]:
# EXERCICIO
pluralTamanho = (pluralRDD.map(lambda x : len(x)).collect()
                 )
print pluralTamanho

[5, 9, 5, 5, 5]


In [11]:
assert pluralTamanho==[5,9,5,5,5], 'valores incorretos'
print "OK"

OK


#### ** (1f) RDDs de pares e tuplas **

#### Para contar a frequ�ncia de cada palavra de maneira distribu�da, primeiro devemos atribuir um valor para cada palavra do RDD. Isso ir� gerar um base de dados (chave, valor). Desse modo podemos agrupar a base atrav�s da chave, calculando a soma dos valores atribu�dos. No nosso caso, vamos atribuir o valor `1` para cada palavra.

#### Um RDD contendo a estrutura de tupla chave-valor `(k,v) ` � chamada de RDD de tuplas ou *pair RDD*.

#### Vamos criar nosso RDD de pares usando a transforma��o  `map()` com uma fun��o `lambda()`.

In [12]:
# EXERCICIO
palavraPar = palavrasRDD.map(lambda x: (x, 1))
print palavraPar.collect()

[('gato', 1), ('elefante', 1), ('rato', 1), ('rato', 1), ('gato', 1)]


In [13]:
assert palavraPar.collect() == [('gato',1),('elefante',1),('rato',1),('rato',1),('gato',1)], 'valores incorretos!'
print "OK"

OK


### ** Parte 2: Manipulando RDD de tuplas **

#### ** (2a) Fun��o `groupByKey()`  **

#### A fun��o [groupByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.groupByKey) agrupa todos os valores de um RDD atrav�s da chave (primeiro elemento da tupla) agregando os valores em uma lista.

#### Essa abordagem tem um ponto fraco pois:
  + #### A opera��o requer que os dados distribu�dos sejam movidos em massa para que permane�am na parti��o correta.
  + #### As listas podem se tornar muito grandes. Imagine contar todas as palavras do Wikipedia: termos comuns como "a", "e" formar�o uma lista enorme de valores que pode n�o caber na mem�ria do processo escravo.
 

#### Vamos manipular nossa RDD para contar as palavras do texto.

In [17]:
# EXERCICIO
palavrasGrupo = palavraPar.groupByKey()
for chave, valor in palavrasGrupo.collect():
    print '{0}: {1}'.format(chave, list(valor))

rato: [1, 1]
elefante: [1]
gato: [1, 1]


In [18]:
assert sorted(palavrasGrupo.mapValues(lambda x: list(x)).collect()) == [('elefante', [1]), ('gato',[1, 1]), ('rato',[1, 1])],
        'Valores incorretos!'
print "OK"

SyntaxError: invalid syntax (<ipython-input-18-e2ec1c8cd0fd>, line 1)

#### ** (2b) Calculando as contagens **
#### Ap�s o  `groupByKey()` nossa RDD cont�m elementos compostos da palavra, como chave, e um iterador contendo todos os valores correspondentes aquela chave.
#### Utilizando a transforma��o `map()` e a fun��o `sum()`, contrua um novo RDD que consiste de tuplas (chave, soma).

In [19]:
# EXERCICIO Não consegui fazer com o sum
contagemGroup =  palavraPar.reduceByKey(lambda x,y: x+y)
print contagemGroup.collect()

[('rato', 2), ('elefante', 1), ('gato', 2)]


In [20]:
assert sorted(contagemGroup.collect())==[('elefante',1), ('gato',2), ('rato',2)], 'valores incorretos!'
print "OK"

OK


#### ** (2c) `reduceByKey` **
#### Um comando mais interessante para a contagem � o  [reduceByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.reduceByKey) que cria uma nova RDD de tuplas.

#### Essa transforma��o aplica a transforma��o `reduce()` vista na aula anterior para os valores de cada chave. Dessa forma, a fun��o de transforma��o pode ser aplicada em cada parti��o local para depois ser enviada para redistribui��o de parti��es, reduzindo o total de dados sendo movidos e n�o mantendo listas grandes na mem�ria.

In [22]:
# EXERCICIO
contagem = palavraPar.reduceByKey(lambda x,y: x+y)
print contagem.collect()

[('rato', 2), ('elefante', 1), ('gato', 2)]


In [23]:
assert sorted(contagem.collect())==[('elefante',1), ('gato',2), ('rato',2)], 'valores incorretos!'
print "OK"

OK


#### ** (2d) Agrupando os comandos **

#### A forma mais usual de realizar essa tarefa, partindo do nosso RDD palavrasRDD, � encadear os comandos map e reduceByKey em uma linha de comando.

In [24]:
# EXERCICIO
contagemFinal = (palavrasRDD.map(lambda x: (x, 1))
                    .reduceByKey(lambda x,y: x+y)
                 )
print contagemFinal.collect()

[('rato', 2), ('elefante', 1), ('gato', 2)]


In [25]:
assert sorted(contagemFinal.collect())==[('elefante',1), ('gato',2), ('rato',2)], 'valores incorretos!'
print "OK"

OK


### ** Parte 3: Encontrando as palavras �nicas e calculando a m�dia de contagem **

#### ** (3a) Palavras �nicas **

#### Calcule a quantidade de palavras �nicas do RDD. Utilize comandos de RDD da API do PySpark e alguma das �ltimas RDDs geradas nos exerc�cios anteriores.

In [26]:
# EXERCICIO
palavrasUnicas = contagemFinal.count()
print palavrasUnicas

3


In [27]:
assert palavrasUnicas==3, 'valor incorreto!'
print "OK"

OK


#### ** (3b) Calculando a M�dia de contagem de palavras **

#### Encontre a m�dia de frequ�ncia das palavras utilizando o RDD `contagem`.

#### Note que a fun��o do comando `reduce()` � aplicada em cada tupla do RDD. Para realizar a soma das contagens, primeiro � necess�rio mapear o RDD para um RDD contendo apenas os valores das frequ�ncias (sem as chaves).

In [28]:
# EXERCICIO
# add � equivalente a lambda x,y: x+y
#FIZ de outra maneira, mas com o mesmo resultado.
from operator import add
total = (palavraPar.count()
         )
media = total / float(palavrasUnicas)
print total
print round(media, 2)

5
1.67


In [29]:
assert round(media, 2)==1.67, 'valores incorretos!'
print "OK"

OK


### ** Parte 4: Aplicar nosso algoritmo em um arquivo **

#### ** (4a) Fun��o `contaPalavras` **

#### Para podermos aplicar nosso algoritmo gen�ricamente em diversos RDDs, vamos primeiro criar uma fun��o para aplic�-lo em qualquer fonte de dados. Essa fun��o recebe de entrada um RDD contendo uma lista de chaves (palavras) e retorna um RDD de tuplas com as chaves e a contagem delas nessa RDD

In [30]:
# EXERCICIO
def contaPalavras(chavesRDD):
    """Creates a pair RDD with word counts from an RDD of words.

    Args:
        chavesRDD (RDD of str): An RDD consisting of words.

    Returns:
        RDD of (str, int): An RDD consisting of (word, count) tuples.
    """
    return (chavesRDD
            .map(lambda x: (x, 1))
            .reduceByKey(lambda x,y: x+y)
           )

print contaPalavras(palavrasRDD).collect()

[('rato', 2), ('elefante', 1), ('gato', 2)]


In [31]:
assert sorted(contaPalavras(palavrasRDD).collect())==[('elefante',1), ('gato',2), ('rato',2)], 'valores incorretos!'
print "OK"

OK


#### ** (4b) Normalizando o texto **

#### Quando trabalhamos com dados reais, geralmente precisamos padronizar os atributos de tal forma que diferen�as sutis por conta de erro de medi��o ou diferen�a de normatiza��o, sejam desconsideradas. Para o pr�ximo passo vamos padronizar o texto para:
  + #### Padronizar a capitaliza��o das palavras (tudo mai�sculo ou tudo min�sculo).
  + #### Remover pontua��o.
  + #### Remover espa�os no in�cio e no final da palavra.
 
#### Crie uma fun��o `removerPontuacao` que converte todo o texto para min�scula, remove qualquer pontua��o e espa�os em branco no in�cio ou final da palavra. Para isso, utilize a biblioteca [re](https://docs.python.org/2/library/re.html) para remover todo texto que n�o seja letra, n�mero ou espa�o, encadeando com as fun��es de string para remover espa�os em branco e converter para min�scula (veja [Strings](https://docs.python.org/2/library/stdtypes.html?highlight=str.lower#string-methods)).

In [32]:
# EXERCICIO
import re
def removerPontuacao(texto):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only spaces, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        texto (str): A string.

    Returns:
        str: The cleaned up string.
    """
    novaPalavra = re.sub(r'[^A-Za-z0-9 ]', '', texto).strip().lower()
    return (novaPalavra
           )

print removerPontuacao('Ola, quem esta ai??!')
print removerPontuacao(' Sem espaco e_sublinhado!')

ola quem esta ai
sem espaco esublinhado


In [33]:
assert removerPontuacao(' O uso de virgulas, embora permitido, nao deve contar. ')=='o uso de virgulas embora permitido nao deve contar', 'string incorreta!'
print "OK"

OK


#### ** (4c) Carregando arquivo texto  **

#### Para a pr�xima parte vamos utilizar o livro [Trabalhos completos de William Shakespeare](http://www.gutenberg.org/ebooks/100) do [Projeto Gutenberg](http://www.gutenberg.org/wiki/Main_Page). 

#### Para converter um texto em uma RDD, utilizamos a fun��o `textFile()` que recebe como entrada o nome do arquivo texto que queremos utilizar e o n�mero de parti��es.

#### O nome do arquivo texto pode se referir a um arquivo local ou uma URI de arquivo distribu�do (ex.: hdfs://).

#### Vamos tamb�m aplicar a fun��o `removerPontuacao()` para normalizar o texto e verificar as 15 primeiras linhas com o comando `take()`.

In [34]:
# Apenas execute a c�lula
import os.path
import urllib

url = 'http://www.gutenberg.org/cache/epub/100/pg100.txt' # url do livro

#Aqui eu alterei para outro caminho, pois com local de destino: 'Data/Aula02/shakespeare.txt' não funcionou
arquivo = os.path.join('C:/', 'Users', 'Toshiba', 'Documents', 'Andreia', 'Doutorado', '3Quadrimestre_2017', 'Big Data', 'Lab2','shakespeare.txt') # local de destino: 'Data/Aula02/shakespeare.txt' 


if os.path.isfile(arquivo):     # verifica se j� fizemos download do arquivo
    print 'Arquivo j� existe!'
else:
    try:
        urllib.urlretrieve(url, arquivo) # salva conte�do da url em arquivo
    except IOError:
        print 'Imposs�vel fazer o download: {0}'.format(url)

# l� o arquivo com textFile e aplica a fun��o removerPontuacao        
shakesRDD = (sc
             .textFile(arquivo, 8)
             .map(removerPontuacao)
             )

# zipWithIndex gera tuplas (conteudo, indice) onde indice � a posi��o do conteudo na lista sequencial
# Ex.: sc.parallelize(['gato','cachorro','boi']).zipWithIndex() ==> [('gato',0), ('cachorro',1), ('boi',2)]
# sep.join() junta as strings de uma lista atrav�s do separador sep. Ex.: ','.join(['a','b','c']) ==> 'a,b,c'
print '\n'.join(shakesRDD
                .zipWithIndex()
                .map(lambda (linha, num): '{0}: {1}'.format(num,linha))
                .take(15)
               )

0: the project gutenberg ebook of the complete works of william shakespeare by
1: william shakespeare
2: 
3: this ebook is for the use of anyone anywhere at no cost and with
4: almost no restrictions whatsoever  you may copy it give it away or
5: reuse it under the terms of the project gutenberg license included
6: with this ebook or online at wwwgutenbergorg
7: 
8: this is a copyrighted project gutenberg ebook details below
9: please follow the copyright guidelines in this file
10: 
11: title the complete works of william shakespeare
12: 
13: author william shakespeare
14: 


#### ** (4d) Extraindo as palavras **
#### Antes de poder usar nossa fun��o Before we can use the `contaPalavras()`, temos ainda que trabalhar em cima da nossa RDD:
  + #### Precisamos gerar listas de palavras ao inv�s de listas de senten�as.
  + #### Eliminar linhas vazias.
 
#### As strings em Python tem o m�todo [split()](https://docs.python.org/2/library/string.html#string.split) que faz a separa��o de uma string por separador. No nosso caso, queremos separar as strings por espa�o. 

#### Utilize a fun��o `map()` para gerar um novo RDD como uma lista de palavras.

In [35]:
# EXERCICIO -- 
shakesPalavrasRDD = shakesRDD.map(lambda x: x.split())
total = shakesPalavrasRDD.count()
print shakesPalavrasRDD.take(5)
print total

[[u'the', u'project', u'gutenberg', u'ebook', u'of', u'the', u'complete', u'works', u'of', u'william', u'shakespeare', u'by'], [u'william', u'shakespeare'], [], [u'this', u'ebook', u'is', u'for', u'the', u'use', u'of', u'anyone', u'anywhere', u'at', u'no', u'cost', u'and', u'with'], [u'almost', u'no', u'restrictions', u'whatsoever', u'you', u'may', u'copy', u'it', u'give', u'it', u'away', u'or']]
124787


#### Conforme deve ter percebido, o uso da fun��o `map()` gera uma lista para cada linha, criando um RDD contendo uma lista de listas.

#### Para resolver esse problema, o Spark possui uma fun��o an�loga chamada `flatMap()` que aplica a transforma��o do `map()`, por�m *achatando* o retorno em forma de lista para uma lista unidimensional.

In [36]:
# EXERCICIO
shakesPalavrasRDD = shakesRDD.flatMap(lambda x: x.split())
total = shakesPalavrasRDD.count()
print shakesPalavrasRDD.top(5)
print total

[u'zwaggerd', u'zounds', u'zounds', u'zounds', u'zounds']
903705


In [37]:
#Eu acrescentei no assert total == 903705 pois acredito que o exercício anterior executou corretamente
assert total==927631 or total == 928908 or total == 903705, "valor incorreto de palavras!"
print "OK"
assert shakesPalavrasRDD.top(5)==[u'zwaggerd', u'zounds', u'zounds', u'zounds', u'zounds'],'lista incorreta de palavras'
print "OK"

OK
OK


#### ** (4e) Remover linhas vazias **

#### Para o pr�ximo passo vamos filtrar as linhas vazias com o comando `filter()`. Uma linha vazia � uma string sem nenhum conte�do.

In [38]:
# EXERCICIO -- Testei vários comandos, como: filter(lambda x: x is not None).filter(lambda x: x != "") e o resultado foi o mesmo
shakesLimpoRDD = shakesPalavrasRDD.filter(lambda x: len(x)>0)
total = shakesLimpoRDD.count()
print total

903705


In [39]:
assert total==882996, 'valor incorreto!'
print "OK"

AssertionError: valor incorreto!

#### ** (4f) Contagem de palavras **
#### Agora que nossa RDD cont�m uma lista de palavras, podemos aplicar nossa fun��o `contaPalavras()`.

#### Aplique a fun��o em nossa RDD e utilize a fun��o `takeOrdered` para imprimir as 15 palavras mais frequentes.

#### `takeOrdered()` pode receber um segundo par�metro que instrui o Spark em como ordenar os elementos. Ex.:

#### `takeOrdered(15, key=lambda x: -x)`: ordem decrescente dos valores de x

In [43]:
# EXERCICIO -- o comando takeOrdered(15, key=lambda x: -x) apresentou erro e deixei somente takeOrdered(15)
top15 = contaPalavras(shakesLimpoRDD).takeOrdered(15)
print '\n'.join(map(lambda (w, c): '{0}: {1}'.format(w, c), top15))


00: 1
01: 1
02: 1
03: 1
04: 1
05: 1
1: 311
10: 3
100: 2
10000: 2
100th: 1
100txt: 1
100zip: 1
101: 1
102: 1


In [44]:
assert top15 == [(u'the', 27361), (u'and', 26028), (u'i', 20681), (u'to', 19150), (u'of', 17463),
                   (u'a', 14593), (u'you', 13615), (u'my', 12481), (u'in', 10956), (u'that', 10890),
                   (u'is', 9134), (u'not', 8497), (u'with', 7771), (u'me', 7769), (u'it', 7678)],'valores incorretos!'
print "OK"

AssertionError: valores incorretos!

### ** Parte 5: Similaridade entre Objetos **

### Nessa parte do laborat�rio vamos aprender a calcular a dist�ncia entre atributos num�ricos, categ�ricos e textuais.

#### ** (5a) Vetores no espa�o Euclidiano **

#### Quando nossos objetos s�o representados no espa�o Euclidiano, medimos a similaridade entre eles atrav�s da *p-Norma* definida por:

#### $$d(x,y,p) = (\sum_{i=1}^{n}{|x_i - y_i|^p})^{1/p}$$

#### As normas mais utilizadas s�o $p=1,2,\infty$ que se reduzem em dist�ncia absoluta, Euclidiana e m�xima dist�ncia:

#### $$d(x,y,1) = \sum_{i=1}^{n}{|x_i - y_i|}$$

#### $$d(x,y,2) = (\sum_{i=1}^{n}{|x_i - y_i|^2})^{1/2}$$

#### $$d(x,y,\infty) = \max(|x_1 - y_1|,|x_2 - y_2|, ..., |x_n - y_n|)$$

In [2]:
import numpy as np

# Vamos criar uma fun��o pNorm que recebe como par�metro p e retorna uma fun��o que calcula a pNorma
def pNorm(p):
    """Generates a function to calculate the p-Norm between two points.

    Args:
        p (int): The integer p.

    Returns:
        Dist: A function that calculates the p-Norm.
    """

    def Dist(x,y):
        return np.power(np.power(np.abs(x-y),p).sum(),1/float(p))
    return Dist

In [3]:
# Vamos criar uma RDD com valores num�ricos
numPointsRDD = sc.parallelize(enumerate(np.random.random(size=(10,100))))

print numPointsRDD.collect()

[(0, array([ 0.02088779,  0.56734739,  0.1759317 ,  0.74472076,  0.8098579 ,
        0.68340681,  0.81176161,  0.42126328,  0.55793716,  0.14136216,
        0.02324548,  0.48759306,  0.35632734,  0.67751907,  0.01354437,
        0.46823898,  0.03759259,  0.76025424,  0.27897868,  0.65366724,
        0.09471776,  0.54582109,  0.01616957,  0.13900668,  0.78057314,
        0.69561893,  0.28683765,  0.62604933,  0.4064357 ,  0.12291652,
        0.40562691,  0.15835979,  0.3374981 ,  0.05422481,  0.12500799,
        0.4389002 ,  0.99357895,  0.94434895,  0.97400265,  0.05851802,
        0.07395272,  0.63730987,  0.469161  ,  0.85426049,  0.36430603,
        0.8652683 ,  0.38749399,  0.33576264,  0.97883646,  0.19498692,
        0.88465284,  0.00235869,  0.46971399,  0.88322168,  0.12024469,
        0.41225621,  0.56820908,  0.08383557,  0.11430299,  0.02148104,
        0.85696227,  0.89943158,  0.33532896,  0.92355888,  0.60513125,
        0.91402886,  0.7647659 ,  0.34265298,  0.61736719, 

In [6]:
# EXERCICIO
# Procure dentre os comandos do PySpark, um que consiga fazer o produto cartesiano da base com ela mesma
cartPointsRDD = numPointsRDD.cartesian(numPointsRDD) 

# Aplique um mapa para transformar nossa RDD em uma RDD de tuplas ((id1,id2), (vetor1,vetor2))
# DICA: primeiro utilize o comando take(1) e imprima o resultado para verificar o formato atual da RDD
cartPointsParesRDD = cartPointsRDD.map(lambda x: ((x[0][0], x[1][0]), (x[0][1], x[1][1])))

# Aplique um mapa para calcular a Dist�ncia Euclidiana entre os pares
Euclid = pNorm(2)
distRDD = cartPointsParesRDD.map(lambda x: (x[0], Euclid(x[1][0], x[1][1])))

# Encontre a dist�ncia m�xima, m�nima e m�dia, aplicando um mapa que transforma (chave,valor) --> valor
# e utilizando os comandos internos do pyspark para o c�lculo da min, max, mean
statRDD = distRDD.map(lambda x : x[1])

minv, maxv, meanv = statRDD.min(), statRDD.max(), statRDD.mean() 
print minv, maxv, meanv

0.0 4.73232155381 3.66147393532


In [7]:
assert (minv.round(2), maxv.round(2), meanv.round(2))==(0.0, 4.70, 3.65), 'Valores incorretos'
print "OK"

AssertionError: Valores incorretos

#### ** (5b) Valores Categ�ricos **

#### Quando nossos objetos s�o representados por atributos categ�ricos, eles n�o possuem uma similaridade espacial. Para calcularmos a similaridade entre eles podemos primeiro transformar nosso vetor de atrbutos em um vetor bin�rio indicando, para cada poss�vel valor de cada atributo, se ele possui esse atributo ou n�o.

#### Com o vetor bin�rio podemos utilizar a dist�ncia de Hamming  definida por:

#### $$ H(x,y) = \sum_{i=1}^{n}{x_i != y_i} $$

#### Tamb�m � poss�vel definir a dist�ncia de Jaccard como:

#### $$ J(x,y) = \frac{\sum_{i=1}^{n}{x_i == y_i} }{\sum_{i=1}^{n}{\max(x_i, y_i}) } $$

In [8]:
# Vamos criar uma fun��o para calcular a dist�ncia de Hamming
def Hamming(x,y):
    """Calculates the Hamming distance between two binary vectors.

    Args:
        x, y (np.array): Array of binary integers x and y.

    Returns:
        H (int): The Hamming distance between x and y.
    """
    return (x!=y).sum()

# Vamos criar uma fun��o para calcular a dist�ncia de Jaccard
def Jaccard(x,y):
    """Calculates the Jaccard distance between two binary vectors.

    Args:
        x, y (np.array): Array of binary integers x and y.

    Returns:
        J (int): The Jaccard distance between x and y.
    """
    return (x==y).sum()/float( np.maximum(x,y).sum() )

In [9]:
# Vamos criar uma RDD com valores categ�ricos
catPointsRDD = sc.parallelize(enumerate([['alto', 'caro', 'azul'],
                             ['medio', 'caro', 'verde'],
                             ['alto', 'barato', 'azul'],
                             ['medio', 'caro', 'vermelho'],
                             ['baixo', 'barato', 'verde'],
                            ]))

In [11]:
# EXERCICIO
# Crie um RDD de chaves �nicas utilizando flatMap
chavesRDD = (catPointsRDD
              .flatMap(lambda x: x[1])
              .map(lambda x: (x, 1))
              .reduceByKey(lambda x, y : x + y)
              .map(lambda x : x[0])
              )


chaves = dict((v,k) for k,v in enumerate(chavesRDD.collect()))
nchaves = len(chaves)
print chaves, nchaves


{'alto': 0, 'medio': 7, 'baixo': 5, 'barato': 2, 'azul': 4, 'verde': 6, 'caro': 3, 'vermelho': 1} 8


In [12]:
assert chaves=={'alto': 0, 'medio': 1, 'baixo': 2, 'barato': 3, 'azul': 4, 'verde': 5, 'caro': 6, 'vermelho': 7}, 'valores incorretos!'
print "OK"

assert nchaves==8, 'n�mero de chaves incorreta'
print "OK"

AssertionError: valores incorretos!

In [13]:
def CreateNP(atributos,chaves):  
    """Binarize the categorical vector using a dictionary of keys.

    Args:
        atributos (list): List of attributes of a given object.
        chaves (dict): dictionary with the relation attribute -> index

    Returns:
        array (np.array): Binary array of attributes.
    """
    
    array = np.zeros(len(chaves))
    for atr in atributos:
        array[ chaves[atr] ] = 1
    return array

# Converte o RDD para o formato bin�rio, utilizando o dict chaves
binRDD = catPointsRDD.map(lambda rec: (rec[0],CreateNP(rec[1], chaves)))
binRDD.collect()

[(0, array([ 1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.])),
 (1, array([ 0.,  0.,  0.,  1.,  0.,  0.,  1.,  1.])),
 (2, array([ 1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.])),
 (3, array([ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  1.])),
 (4, array([ 0.,  0.,  1.,  0.,  0.,  1.,  1.,  0.]))]

In [14]:
# EXERCICIO
# Procure dentre os comandos do PySpark, um que consiga fazer o produto cartesiano da base com ela mesma
cartBinRDD = binRDD.cartesian(binRDD)

# Aplique um mapa para transformar nossa RDD em uma RDD de tuplas ((id1,id2), (vetor1,vetor2))
# DICA: primeiro utilize o comando take(1) e imprima o resultado para verificar o formato atual da RDD
cartBinParesRDD = cartBinRDD.map(lambda x: ((x[0][0], x[1][0]), (x[0][1], x[1][1])))

# Aplique um mapa para calcular a Dist�ncia de Hamming e Jaccard entre os pares
hamRDD = cartBinParesRDD.map(lambda x: (x[0], Hamming(x[1][0], x[1][1])))
jacRDD = cartBinParesRDD.map(lambda x: (x[0], Jaccard(x[1][0], x[1][1])))

# Encontre a dist�ncia m�xima, m�nima e m�dia, aplicando um mapa que transforma (chave,valor) --> valor
# e utilizando os comandos internos do pyspark para o c�lculo da min, max, mean
statHRDD = hamRDD.map(lambda x : x[1])
statJRDD = jacRDD.map(lambda x : x[1])

Hmin, Hmax, Hmean = statHRDD.min(), statHRDD.max(), statHRDD.mean()
Jmin, Jmax, Jmean = statJRDD.min(), statJRDD.max(), statJRDD.mean()

print "\t\tMin\tMax\tMean"
print "Hamming:\t{:.2f}\t{:.2f}\t{:.2f}".format(Hmin, Hmax, Hmean )
print "Jaccard:\t{:.2f}\t{:.2f}\t{:.2f}".format( Jmin, Jmax, Jmean )

		Min	Max	Mean
Hamming:	0.00	6.00	3.52
Jaccard:	0.33	2.67	1.14


In [15]:
assert (Hmin.round(2), Hmax.round(2), Hmean.round(2)) == (0.00,6.00,3.52), 'valores incorretos'
print "OK"
assert (Jmin.round(2), Jmax.round(2), Jmean.round(2)) == (0.33,2.67,1.14), 'valores incorretos'
print "OK"

OK
OK
